In [272]:
from utils import find_needed_for,rows,find_by_name
import random
from heapq import heappush, heappop
import json

In [273]:
def print_storage_not_nan(storage):
    print("=====Storage=====")
    for k,v in storage.items():
        if v != 0:
            print(f"{k}: {v}")
    print("=================")

In [274]:
storage = {}
for row in rows:
    storage[row['Name']] = 1

In [275]:
sources_key = set()
for row in rows:
    sources_key.add(row['Source'])

In [276]:
sources = {}
for key in sources_key:
    sources[key] = [
        {
            'status':'idle',
            'product':None,
            'start_time':None,
            'end_time':None,
        }
    ]

for _ in range(20):
    sources['Field'].append( {
            'status':'idle',
            'product':None,
            'start_time':None,
            'end_time':None,
        })

In [277]:
tasks = []
test = find_needed_for('Bread')

def add_to_queue(product):
    for need in product['Needs']:
        add_to_queue(need)
    for _ in range(product['Count']):
        tasks.append(product['Name'])
add_to_queue(test)

In [278]:
time_queue = [0]

while len(time_queue) > 0:
    time = heappop(time_queue)
    print('Time:',time)
    print(tasks)
    for source_name in sources:
        for i,s in enumerate(sources[source_name]):
            if s['end_time'] == time:
                if source_name=='Field':
                    storage[s['product']] += 2
                else:
                    storage[s['product']] += 1
                s['status'] = 'idle'
                s['product'] = None
                s['start_time'] = None
                s['end_time'] = None
    # print_storage_not_nan(storage)
    for i,task in enumerate(tasks.copy()):
        task = find_by_name(task)

        if_have_enough_materials = True
        for need in task['Needs']:
            key,value = list(need.items())[0]
            if storage[key] < value:
                if_have_enough_materials = False
                break
        print(task['Name'],i,'if_have_enough_materials',if_have_enough_materials)
        if not if_have_enough_materials:
            continue
        if_have_enough_sources = False
        source = task['Source']
        for s in sources[source]:
            if s['status'] == 'idle':
                if_have_enough_sources = True
                break
        print(task['Name'],i,'if_have_enough_sources',if_have_enough_sources)
        if not if_have_enough_sources:
            continue
        print('Schedule:',task['Name'])
        s['status'] = 'busy'
        s['product'] = task['Name']
        s['start_time'] = time
        s['end_time'] = time + task['Time']
        for need in task['Needs']:
            key,value = list(need.items())[0]
            storage[key] -= value
        if s['end_time'] not in time_queue:
            heappush(time_queue,s['end_time'])
        tasks.remove(task['Name'])
    # print_storage_not_nan(storage)

Time: 0
['Wheat', 'Wheat', 'Wheat', 'Bread']
Wheat 0 if_have_enough_materials True
Wheat 0 if_have_enough_sources True
Schedule: Wheat
Wheat 1 if_have_enough_materials False
Wheat 2 if_have_enough_materials False
Bread 3 if_have_enough_materials False
Time: 120
['Wheat', 'Wheat', 'Bread']
Wheat 0 if_have_enough_materials True
Wheat 0 if_have_enough_sources True
Schedule: Wheat
Wheat 1 if_have_enough_materials True
Wheat 1 if_have_enough_sources True
Schedule: Wheat
Bread 2 if_have_enough_materials False
Time: 240
['Bread']
Bread 0 if_have_enough_materials True
Bread 0 if_have_enough_sources True
Schedule: Bread
Time: 540
[]
